In [1]:
csv_path = '//'

In [ ]:
#Install the package
!pip install py-feat

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import libraries/packages
import os
import pandas as pd
from feat import Detector
import requests
import cv2
from feat.utils.io import read_feat
import numpy as np



detector = Detector(device='cuda')

In [ ]:
#helper functions
def make_dir(csv_path):
  csv_name = os.path.splitext(os.path.basename(csv_path))[0]
  #for the images folder
  image_folder = '/content/drive/MyDrive/Colab Notebooks/Emotions/Scripts/images'
  images_path = os.path.join(image_folder, "images-"+csv_name)
  if not os.path.exists(images_path):
    os.mkdir(images_path)
  #for the fex folder
  fex_folder =  '/content/drive/MyDrive/Colab Notebooks/Emotions/Scripts/FexData'
  fex_path = os.path.join(fex_folder, "Fex-"+csv_name)
  if not os.path.exists(fex_path):
    os.mkdir(fex_path)
  return images_path,fex_path,csv_name

def download_images(csv_path,image_path,batch_size):
  df = pd.read_csv(csv_path)
  subdf = df[["id","media_url"]]
  total = len(subdf)

  subfolder_count = total // batch_size + 1

  for n in range(subfolder_count):

    subfolder = os.path.join(image_path,f'file {n+1}')
    if not os.path.exists(subfolder):
      os.mkdir(subfolder)
    slicedf= subdf[n*batch_size : (n+1)*batch_size]

    for img in slicedf.iloc:
      url = img["media_url"]

      try:
        r = requests.get(url)
        if not os.path.exists(os.path.join(subfolder, str(img["id"]))):
          with open(os.path.join(subfolder, str(img["id"])), 'wb') as outfile:
            outfile.write(r.content)
      except:
        pass
  return subfolder_count

def extract_features(images_path,fex_path,subfolder_count):
  for foldernum in range(1,subfolder_count+1):
    image_subfolder_path = os.path.join(images_path,"file "+str(foldernum))
    fex_outfile_path = os.path.join(fex_path,"file "+str(foldernum))
    if not os.path.exists(fex_outfile_path):
      os.mkdir(fex_outfile_path)
    for filename in os.listdir(image_subfolder_path):
      file_path = os.path.join(image_subfolder_path, filename)
      if not os.path.isfile(os.path.join(fex_outfile_path, str(filename)+ ".csv")):
        # Try reading file as image
        try:
          fex = detector.detect_image(file_path)
          fex.to_csv(os.path.join(fex_outfile_path, str(filename)+ ".csv"), index=False)
        except:
          pass
      else:
        continue

def ClassifyEmotion(input_prediction):
  image_id = os.path.splitext(os.path.basename(input_prediction["input"][0]))[0]
  emotionList = []
  emotionValueList= []
  for i in range(len(input_prediction)): 
    if not input_prediction.drop(["input","frame"],axis=1).iloc[i].isnull().all():
      dominantEmotion = input_prediction.iloc[i].emotions.sort_values(ascending = False).keys()[0] 
      dominantEmotionValue = input_prediction.iloc[i].emotions.sort_values(ascending = False)[0]
      emotionList.append(dominantEmotion)
      emotionValueList.append(dominantEmotionValue)
      dominantEmotionOverall = max(emotionList,key=emotionList.count)
    else:
      dominantEmotionOverall = None
  image_id_list = [image_id] * len(emotionList)
  return image_id_list,emotionList,emotionValueList


def create_df(fex_folder,fex_folder_num):
  image_id = []
  emotionDom = []
  emotionList = []
  emotionVals= []
  image_id = []
  for filenum in range(1,fex_folder_num+1):
    print('On file ' + f"{filenum}")
    subfolder = os.path.join(fex_folder,f"file {filenum}")
    i = 0
    for fexfile in os.listdir(subfolder):  # for each image
      input_prediction = read_feat(os.path.join(subfolder, fexfile))
      id,emotion,emotionValue = new_ClassifyEmotion(input_prediction)  # changes ClassifyEmotion to reflect new coding
      image_id+=id
      emotionList+=emotion
      emotionVals+=emotionValue
      i += 1
      if i % 1000 == 0:
        print(f"Processed {i} images.")

  #create df
  dict = {'id': image_id, 'emotionList': emotionList,'emotionVals':emotionVals}

  return pd.DataFrame(dict)


def new_ClassifyEmotion(input_prediction):
  image_id = os.path.splitext(os.path.basename(input_prediction["input"][0]))[0]
  emotionList = []
  emotionValueList= []
  for i in range(len(input_prediction)): 
    if not input_prediction.drop(["input","frame"],axis=1).iloc[i].isnull().all():  # skips any row where a face emotion is not estimated
      emotions = input_prediction.iloc[i].emotions.sort_values(ascending=False).keys()  # emotion names
      emotionValues = input_prediction.iloc[i].emotions.sort_values(ascending = False)  # emotion values
      emotionList.append(emotions)
      emotionValueList.append(emotionValues)
    else:
      pass

  image_id_list = [image_id] * len(emotionList) # match tweet id to list of emotions
  return image_id_list,emotionList,emotionValueList



In [ ]:

def process_csv(csv_path,batch_size,subfolder_count):
  df = create_df(fex_folder,fex_folder_num=subfolder_count) 
  print("==========Creating CSV!==========")
  data = pd.read_csv(csv_path)
  df['id']=df['id'].astype(np.int64)
  merged_df = df.merge(data, how='right', on='id')
  merged_df.to_csv(f'{os.path.dirname(csv_path)}/{csv_name}-Emotions.csv', encoding='utf-8', index=False) 
  print("==========CSV Created!==========")
  return print("Done!")


batch_size = 10000
csv_path = 'csvpath'
csv_name = os.path.splitext(os.path.basename(csv_path))[0]
subfolder_count = 1
fex_folder =  'fexpath'
fex_path = os.path.join(fex_folder, "Fex-"+csv_name)
fex_folder = fex_path  

process_csv(csv_path=csv_path, batch_size=batch_size)


,id,duplicate_id,imgpath,protest,violence,state_violence,protester_violence,sign,photo,fire,...,raceSEAsian,faces0_2,faces3_9,faces10_19,faces20_29,faces30_39,faces40_49,faces50_59,faces60_69,faces70plus
0,1190786238426599425,750.0,/media/alex/protests/events/IQ2019-10-01_00_20...,0.673626,0.403902,0.001253,0.001734,0.890752,0.009797,0.003212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1188389350096035840,434.0,/media/alex/protests/events/IQ2019-10-01_00_20...,0.755637,0.225750,0.001508,0.006365,0.983822,0.057794,0.005496,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1204129060919685126,1299.0,/media/alex/protests/events/IQ2019-10-01_00_20...,0.879376,0.592216,0.136439,0.176236,0.505921,0.021374,0.078124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1202633968484585474,1185.0,/media/alex/protests/events/IQ2019-10-01_00_20...,0.708588,0.407706,0.013744,0.032396,0.635046,0.055930,0.029376,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1179143039429890048,55.0,/media/alex/protests/events/IQ2019-10-01_00_20...,0.711804,0.450279,0.017555,0.080173,0.493333,0.044507,0.083209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# RUN THIS TO TEST
import time
time_start = time.time()
process_csv(csv_path,10000)
time_end = time.time()
print("The process took: ",time_end-time_start," seconds")

==========Folders Created!==========
==========Images Downloaded!==========


  0%|          | 0/1 [00:00<?, ?it/s]


==========Features Extracted!==========
==========Creating CSV!==========
==========CSV Created!==========
Done!
The process took:  689.373782157898  seconds
